In [19]:
import sagemaker.huggingface

In [20]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::707684582322:role/service-role/AmazonSageMaker-ExecutionRole-20191024T163188
sagemaker bucket: sagemaker-eu-west-1-707684582322
sagemaker session region: eu-west-1


In [21]:
import boto3
import os

training_job_name = "huggingface-finetune-twitter2023-03-16--2023-03-16-01-39-16-251"
s3_model_location = 's3://{}/{}/output/model.tar.gz'.format(sagemaker_session_bucket, training_job_name)
s3_model_location

's3://sagemaker-eu-west-1-707684582322/huggingface-finetune-twitter2023-03-16--2023-03-16-01-39-16-251/output/model.tar.gz'

In [22]:
!pygmentize ./scripts/inference_nlp_bert_sm_compatible.py

import numpy as np
import torch 
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel, AutoConfig

import torch.nn as nn
import torch.nn.functional as F

import argparse
import logging
import os
import random
import sys

from datetime import datetime

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


TOKENIZER_MODEL = "bert-base-multilingual-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model = "bert-base-multilingual-uncased"

# PyTorch models inherit from torch.nn.Module
class SentenceMultiClassClassifier(nn.Module):
    def __init__(self,number_class, pretrained_model):
        super(SentenceMultiClassClassifier, self).__init__()
        self.number_class = number_class
        #self.pretrained = AutoModel.from_pretrained(pretrained_model).to(device)
        self.pretrained = AutoModel.from_pretrained(pretrained_model,config=AutoConfig.from_pretrai

In [23]:
instance_type = "ml.g4dn.xlarge"

In [24]:
s3_model_location

's3://sagemaker-eu-west-1-707684582322/huggingface-finetune-twitter2023-03-16--2023-03-16-01-39-16-251/output/model.tar.gz'

In [25]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_model_location, # path to your model and script
   entry_point="./scripts/inference_nlp_bert_sm_compatible.py",
   source_dir = './scripts',
   role=role,
   transformers_version="4.6",  # transformers version used
   pytorch_version="1.7",        # pytorch version used
   py_version='py36',            # python version used
)

In [26]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type=instance_type
)

------!

In [39]:
predictions = predictor.predict("I am so happy about this situation")

In [40]:
predictions

[[0.24477840960025787,
  0.13813115656375885,
  0.25555047392845154,
  0.17213194072246552,
  0.4122581481933594,
  0.11252996325492859,
  0.3421306908130646,
  0.151715949177742,
  0.33474868535995483,
  0.0483432300388813,
  0.052165474742650986]]

In [41]:
#predictor.delete_endpoint()

## Inference with SDK Boto3

In [42]:
import boto3
import json

In [43]:
runtime = boto3.client("sagemaker-runtime")

In [46]:
endpoint_name = "huggingface-pytorch-inference-2023-03-16-06-10-43-495"

In [47]:
input_json = "I am so happy about this situation"

In [76]:
def prediction_with_endpoint(endpoint_name,runtime,text):
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(input_json),
        ContentType="application/json",
    )
    results = response["Body"].read()
    return json.loads(results)

In [78]:
print(prediction_with_endpoint(endpoint_name, runtime, input_json))

[[0.24477840960025787, 0.13813115656375885, 0.25555047392845154, 0.17213194072246552, 0.4122581481933594, 0.11252996325492859, 0.3421306908130646, 0.151715949177742, 0.33474868535995483, 0.0483432300388813, 0.052165474742650986]]


## Evaluation on twitter dataset

In [68]:
import pandas as pd 
import numpy as np

In [69]:
df = pd.read_csv("./dataset/sem_eval_2018_task_1_test.csv")

In [70]:
label_name = [label for label in df.columns if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(label_name)}
id2label

{0: 'anger',
 1: 'anticipation',
 2: 'disgust',
 3: 'fear',
 4: 'joy',
 5: 'love',
 6: 'optimism',
 7: 'pessimism',
 8: 'sadness',
 9: 'surprise',
 10: 'trust'}

In [90]:
label_index = df.iloc[:,2:].astype(int)
label_index.head(5)

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,1,1,0,0,0,0,1,0,0,0,1
1,0,0,1,0,0,0,0,0,0,0,0
2,1,0,1,0,0,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,1,0,0


In [91]:
text_tweet = df.loc[:,["Tweet"]].values.tolist()

In [92]:
text_tweet[0]

['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell']

In [93]:
results = prediction_with_endpoint(endpoint_name, runtime, text_tweet[0])
results

[[0.24477840960025787,
  0.13813115656375885,
  0.25555047392845154,
  0.17213194072246552,
  0.4122581481933594,
  0.11252996325492859,
  0.3421306908130646,
  0.151715949177742,
  0.33474868535995483,
  0.0483432300388813,
  0.052165474742650986]]

In [94]:
def proba_to_label(prediction_proba,id2label,threshold):
    prediction_label = []
    for index in range(len(prediction_proba)):
        if prediction_proba[index] > threshold:
            prediction_label.append(id2label[index])
            
    return prediction_label

In [95]:
print(proba_to_label(results[0],id2label,0.3))

['joy', 'optimism', 'sadness']


### Prediction over entire dataset

In [96]:
predictions_proba = np.zeros_like(label_index,dtype=float)

#### real-time endpoints that make one prediction at a time, over http 

In [97]:
for index in range(len(text_tweet)):
    result = prediction_with_endpoint(endpoint_name, runtime, text_tweet[index])
    result = np.squeeze(np.array(result), axis=0)    
    predictions_proba[index,:] = result    

In [98]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [99]:
def multi_label_metrics(labels, predictions_proba, threshold = 0.5):
    y_true = labels

    predictions = np.zeros_like(labels)
    predictions[np.where(predictions_proba >= threshold)] = 1
    
    y_pred = predictions
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1_micro_average = f1_score(y_true, y_pred, average='micro')
    
    metrics = {'precision': precision,
               'recall': recall,
               'f1': f1_micro_average}
    return metrics

In [100]:
metrics = multi_label_metrics(label_index, predictions_proba, threshold = 0.3)

In [101]:
metrics

{'precision': 0.3625856602229723,
 'recall': 0.4505019697547338,
 'f1': 0.4017907741131134}